In [343]:
import pandas as pd
import numpy as np
import math
Ratings=pd.read_csv("ratings.csv")
Movies=pd.read_csv("movies.csv")
Tags=pd.read_csv("tags.csv")

In [344]:
Ratings.iloc[np.where(Ratings.userId ==12882)].head()

,userId,movieId,rating,timestamp
0,12882,1,4.0,1147195252
1,12882,32,3.5,1147195307
2,12882,47,5.0,1147195343
3,12882,50,5.0,1147185499
4,12882,110,4.5,1147195239


In [345]:
Movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [333]:
Tags.iloc[np.where(Tags.movieId==3916)].head()

,movieId,userId,tag,timestamp
0,3916,12882,sports,1147195545
4322,3916,12778,sports,1137526967
6513,3916,130819,Denzel Washington,1220315397
7026,3916,95552,based on a true story,1330489043
7027,3916,95552,football,1330489048


In [22]:
a=math.log10(len(np.unique(Tags[‘movieId’])))
DF[‘IDF’]=a-np.log10(DF[‘tag_count_DF’])
TF = pd.merge(TF,DF,on = ‘tag’, how = ‘left’, sort = False)
TF[‘TF-IDF’]=TF[‘tag_count_TF’]*TF[‘IDF’]

SyntaxError: invalid syntax (<ipython-input-22-58359bbecf05>, line 2)

In [37]:
#計算每一個Tag在每一個movie上的次數，轉換成這個關鍵字對於此電影的TF-IDF
TF= Tags.groupby(['movieId','tag'], as_index = False, sort = False).count().rename(columns = {'userId': 'tag_count_TF'})[['movieId','tag','tag_count_TF']]
Tag_distinct = Tags[['tag','movieId']].drop_duplicates()
DF =Tag_distinct.groupby(['tag'], as_index = False, sort = False).count().rename(columns = {'movieId': 'tag_count_DF'})[['tag','tag_count_DF']]
a=math.log10(len(np.unique(Tags['movieId'])))
DF['IDF']=a-np.log10(DF['tag_count_DF'])
TF = pd.merge(TF,DF,on = 'tag', how = 'left', sort = False)
TF['TF-IDF']=TF['tag_count_TF']*TF['IDF']

In [335]:
#TF.head()

In [48]:
Vect_len=TF[['movieId','TF-IDF']]
Vect_len['TF-IDF-Sq']=Vect_len['TF-IDF']**2
Vect_len =Vect_len.groupby(['movieId'], as_index = False, sort = False).sum().rename(columns = {
        'TF-IDF-Sq': 'TF-IDF-Sq-sum'})[['movieId','TF-IDF-Sq-sum']]
Vect_len['vect_len'] = np.sqrt(Vect_len[['TF-IDF-Sq-sum']].sum(axis=1))
TF = pd.merge(TF,Vect_len,on = 'movieId', how = 'left', sort = False)
TF['TAG_WT']=TF['TF-IDF']/TF['vect_len']

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [53]:
Ratings_filter=Ratings[Ratings['rating']>=3.5]
distinct_users=np.unique(Ratings['userId'])
user_tag_pref=pd.DataFrame()

In [72]:
i=1
for user in distinct_users:
    if i%20==0:
        print'user:', i , 'out of:', len(distinct_users)
    user_data= Ratings_filter[Ratings_filter['userId']==user]
    user_data = pd.merge(TF,user_data,on = 'movieId', how = 'inner', sort = False)
    user_data1 = user_data.groupby(['tag'], as_index = False, sort = False).sum().rename(columns = {
                'TAG_WT': 'tag_pref'})[['tag','tag_pref']]
    user_data1['user']=user
    user_tag_pref = user_tag_pref.append(user_data1, ignore_index=True)
    i=i+1

user: 20 out of: 862
user: 40 out of: 862
user: 60 out of: 862
user: 80 out of: 862
user: 100 out of: 862
user: 120 out of: 862
user: 140 out of: 862
user: 160 out of: 862
user: 180 out of: 862
user: 200 out of: 862
user: 220 out of: 862
user: 240 out of: 862
user: 260 out of: 862
user: 280 out of: 862
user: 300 out of: 862
user: 320 out of: 862
user: 340 out of: 862
user: 360 out of: 862
user: 380 out of: 862
user: 400 out of: 862
user: 420 out of: 862
user: 440 out of: 862
user: 460 out of: 862
user: 480 out of: 862
user: 500 out of: 862
user: 520 out of: 862
user: 540 out of: 862
user: 560 out of: 862
user: 580 out of: 862
user: 600 out of: 862
user: 620 out of: 862
user: 640 out of: 862
user: 660 out of: 862
user: 680 out of: 862
user: 700 out of: 862
user: 720 out of: 862
user: 740 out of: 862
user: 760 out of: 862
user: 780 out of: 862
user: 800 out of: 862
user: 820 out of: 862
user: 840 out of: 862
user: 860 out of: 862


In [262]:
TF['sq_TAG_WT'] = TF['TAG_WT']*TF['TAG_WT']
get_1=pd.merge(TF,user_tag_pref,on='tag',how='left',sort=False)
get_1['tag_pref']=get_1['tag_pref'].fillna(0)
get_1['tag_value'] = get_1['TAG_WT']*get_1['tag_pref']
get_movie_pre_score = get_1.groupby(['movieId','user'])[['sq_TAG_WT']].sum().rename(columns = {
                'sq_TAG_WT': 'sq_TAG_Val'}).reset_index()

In [263]:
get_movie_pre_score['TAG_WT_VAL'] = np.sqrt(get_movie_pre_score['sq_TAG_Val'])

In [264]:
user_tag_pref['sq_tag_pref'] = np.square(user_tag_pref['tag_pref'])
get_user_score=user_tag_pref.groupby(['user'])[['sq_tag_pref']].sum().rename(columns={'sq_tag_pref':'sq_tag_pre_val'}).reset_index()
get_user_score['tag_pre_val'] = np.sqrt(get_user_score['sq_tag_pre_val'])

In [265]:
user_movie_rating = get_1.groupby(['user','movieId'])[['tag_value']].sum().rename(columns = {
                'tag_value': 'Rating'}).reset_index()
user_movie_rating = pd.merge(user_movie_rating,get_movie_pre_score,on=['user','movieId'],how='inner')
user_movie_rating = pd.merge(user_movie_rating,get_user_score,on='user',how='inner')

In [273]:
#user_movie_rating['TAG_WT_VAL']*
user_movie_rating['Rating'] =user_movie_rating['Rating']/(user_movie_rating['tag_pre_val'])
tag_merge_all=user_movie_rating[['user','movieId','Rating']]

In [269]:
"""
原文作法
tag_merge_all=pd.DataFrame()
i=1
for user in distinct_users[1:100]:
    user_tag_pref_all= user_tag_pref[user_tag_pref['user']==user]
    distinct_movies = np.unique(TF['movieId'])
    j=1
    for movie in distinct_movies:
        if j%300==0:
            print 'movie: ', j , 'out of: ', len(distinct_movies) , 'with user: ', i , 'out of: ', len(distinct_users)
            
        TF_Movie= TF[TF['movieId']==movie]
        tag_merge = pd.merge(TF_Movie,user_tag_pref_all,on = 'tag', how = 'left', sort = False)
        tag_merge['tag_pref']=tag_merge['tag_pref'].fillna(0)
        tag_merge['tag_value']=tag_merge['TAG_WT']*tag_merge['tag_pref']
        TAG_WT_val=np.sqrt(np.sum(np.square(tag_merge['TAG_WT']), axis=0))
        tag_pref_val=np.sqrt(np.sum(np.square(user_tag_pref_all['tag_pref']), axis=0))
        tag_merge_final = tag_merge.groupby(['user','movieId'])[['tag_value']].sum().rename(columns = {
                'tag_value': 'Rating'}).reset_index()
        tag_merge_final['Rating']=tag_merge_final['Rating']/(TAG_WT_val*tag_pref_val)
        tag_merge_all = tag_merge_all.append(tag_merge_final, ignore_index=True)
        j=j+1
    i=i+1
tag_merge_all=tag_merge_all.sort_index(by=['user','Rating']).reset_index()
 """

"\n\ntag_merge_all=pd.DataFrame()\ni=1\nfor user in distinct_users[1:100]:\n    user_tag_pref_all= user_tag_pref[user_tag_pref['user']==user]\n    distinct_movies = np.unique(TF['movieId'])\n    j=1\n    for movie in distinct_movies:\n        if j%300==0:\n            print 'movie: ', j , 'out of: ', len(distinct_movies) , 'with user: ', i , 'out of: ', len(distinct_users)\n            \n        TF_Movie= TF[TF['movieId']==movie]\n        tag_merge = pd.merge(TF_Movie,user_tag_pref_all,on = 'tag', how = 'left', sort = False)\n        tag_merge['tag_pref']=tag_merge['tag_pref'].fillna(0)\n        tag_merge['tag_value']=tag_merge['TAG_WT']*tag_merge['tag_pref']\n        TAG_WT_val=np.sqrt(np.sum(np.square(tag_merge['TAG_WT']), axis=0))\n        tag_pref_val=np.sqrt(np.sum(np.square(user_tag_pref_all['tag_pref']), axis=0))\n        tag_merge_final = tag_merge.groupby(['user','movieId'])[['tag_value']].sum().rename(columns = {\n                'tag_value': 'Rating'}).reset_index()\n       

In [283]:
#tag_merge_all.sort_values(by=['user','Rating'],ascending=[True,False]).reset_index().drop('index',axis=1)

In [339]:
#tag_merge_all.head()

# Assignment with Weighted User-Profile

In [285]:
Ratings.head()

,userId,movieId,rating,timestamp
0,12882,1,4.0,1147195252
1,12882,32,3.5,1147195307
2,12882,47,5.0,1147195343
3,12882,50,5.0,1147185499
4,12882,110,4.5,1147195239


In [294]:
#minus average of user's personalized rating-score
personal_avg_score = Ratings.groupby(['userId'])[['rating']].mean().reset_index().rename(columns={'rating':'avg_rating'})

In [307]:
Rating_Avg = pd.merge(Ratings,personal_avg_score,on = 'userId',how = 'inner')
Rating_Avg['weight']=Rating_Avg.rating - Rating_Avg.avg_rating
user_movie_weight = pd.merge(TF,Rating_Avg,on='movieId',how='inner',sort=False)
user_movie_weight['TAG_WT_WTD']=user_movie_weight['TAG_WT']*user_movie_weight['weight']

In [320]:
user_tag_pref2 = user_movie_weight.groupby(['tag','userId'],as_index=False, sort =False).sum().rename(columns = {
        'TAG_WT_WTD': 'tag_pref'})[['tag','userId','tag_pref']]
 

In [324]:
get_2=pd.merge(TF,user_tag_pref2,on='tag',how='left',sort=False)
get_2['tag_pref']=get_2['tag_pref'].fillna(0)
get_2['tag_value'] = get_2['TAG_WT']*get_1['tag_pref']
get_movie_pre_score2 = get_2.groupby(['movieId','userId'])[['sq_TAG_WT']].sum().rename(columns = {
                'sq_TAG_WT': 'sq_TAG_Val'}).reset_index()

In [325]:
get_movie_pre_score2['TAG_WT_VAL'] = np.sqrt(get_movie_pre_score2['sq_TAG_Val'])
user_tag_pref2['sq_tag_pref'] = np.square(user_tag_pref2['tag_pref'])
get_user_score2=user_tag_pref2.groupby(['userId'])[['sq_tag_pref']].sum().rename(columns={'sq_tag_pref':'sq_tag_pre_val'}).reset_index()
get_user_score2['tag_pre_val'] = np.sqrt(get_user_score2['sq_tag_pre_val'])

In [326]:
user_movie_rating2 = get_2.groupby(['userId','movieId'])[['tag_value']].sum().rename(columns = {
                'tag_value': 'Rating'}).reset_index()
user_movie_rating2 = pd.merge(user_movie_rating2,get_movie_pre_score2,on=['userId','movieId'],how='inner')
user_movie_rating2 = pd.merge(user_movie_rating2,get_user_score2,on='userId',how='inner')

In [328]:
#user_movie_rating['TAG_WT_VAL']*
user_movie_rating2['Rating'] =user_movie_rating2['Rating']/(user_movie_rating2['tag_pre_val'])
tag_merge_all2=user_movie_rating2[['userId','movieId','Rating']]

In [330]:
#tag_merge_all2

In [342]:
#tag_merge_all2.head()